In [930]:
import numpy as np
import scipy.stats as scps
from scipy.special.orthogonal import ps_roots
import scipy.interpolate as scin
from copy import *
%load_ext autoreload

%autoreload 2




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [931]:
from ret import util, mutil, imutil, income, budget, mbudget, value_function, budget_old, chpr, logsum, egm_step,chop, polyline, diff, aux_function, interpolate, upper_envelope

# Model parameters (default)

In [932]:
# Number of periods (fist period is t=1) 
Tbar = 25

# Number of grid points over assets
ngridm = 500

# Maximum level of assets
mmax = 50

# Number of quadrature points used in calculation of expectations
n_quad_points = 5

# Number of simulations
nsims = 10

# Interval of the initial wealth
init = [10, 30]

# Interest rate
r = 0.05

# Discount factor
beta = 0.95

# Standard deviation of log-normally distributed income shocks
sigma = 0.00

# Disutility of work
duw = 0.35

# CRRA coefficient (log utility if ==1)
theta = 1.95

# Careful with the coefficients here -- original code had the polynomial
# Coded as a + b * x - c * x**2 ... (note the crazy minus)
coeffs_age_poly = np.array([0.75, 0.04, -0.0002])

# Consumption floor (safety net in retirement)
cfloor = 0.001

# Scale of the EV taste shocks 
lambda_ = 2.2204e-16

In [933]:
# Initialize grids
quadstnorm = scps.norm.ppf(ps_roots(n_quad_points)[0])
quadw = ps_roots(n_quad_points)[1]

In [934]:
savingsgrid = np.linspace(0, mmax, ngridm)

In [935]:
# We are currently facing a problem since our result arrays do not have the same length. 
# Instead of a multidimensional array we should use a list in which we store the different arrays.
# What would this mean for our functions? 

In [950]:
# Set up list containers
policy = [[np.full((ngridm + 1, 2), np.nan) for k in range(2)] for i in range(Tbar)]
value = [[np.full((ngridm + 1, 2), np.nan) for k in range(2)] for i in range(Tbar)]
# Handling of last period and first elements
# policy
for k in range(Tbar):
    for choice in range(2):
        value[k][choice][0, 0] = 0.00
        policy[k][choice][0, :] = 0.00

policy[Tbar - 1][0][1:, 0] = deepcopy(savingsgrid)
policy[Tbar-1][1][1:, 0] = deepcopy(savingsgrid)
policy[Tbar-1][0][1:, 1] = deepcopy(policy[Tbar-1][0][1:, 0])
policy[Tbar-1][1][1:, 1] = deepcopy(policy[Tbar-1][1][1:, 0])
# value
value[Tbar-1][0][2:, 0] = util(policy[Tbar-1][0][2:, 0], 0, theta, duw)
value[Tbar-1][0][2:, 1] = util(policy[Tbar-1][1][2:, 0], 1, theta, duw)
value[Tbar-1][1][2:, 0] = util(policy[Tbar-1][0][2:, 0], 0, theta, duw)
value[Tbar-1][1][2:, 1] = util(policy[Tbar-1][1][2:, 0], 1, theta, duw)

value[Tbar -1][0][0:2] = 0.00
value[Tbar -1][1][0:2] = 0.00

# The time and the choice dimension are now extracted from our array format. 
# Instead we are using a list for saving the results which allows us to alter the legth of the arrays


In [951]:
for period in [23, 22, 21, 20, 19, 18]:
    value, policy = egm_step(value, policy, savingsgrid, quadstnorm, period, Tbar, ngridm, cfloor, n_quad_points, r, coeffs_age_poly, theta, duw, beta, lambda_, sigma, quadw)

In [952]:
import pickle
with open('m0_value.pkl', 'rb') as file : 
    m0_value = pickle.load(file)

with open('m0_policy.pkl', 'rb') as file : 
    m0_policy = pickle.load(file)
    

for point in [23, 22, 21, 20, 19]:
    np.testing.assert_almost_equal(m0_value[:, 0, 1, point].T[~np.isnan(m0_value[:, 0, 1, point])], value[point][1].T[0])
    np.testing.assert_almost_equal(m0_value[:, 1, 1, point].T[~np.isnan(m0_value[:, 1, 1, point])], value[point][1].T[1])
    np.testing.assert_almost_equal(m0_policy[:, 0, 1, point].T[~np.isnan(m0_policy[:, 0, 1, point])], policy[point][1].T[0])
    np.testing.assert_almost_equal(m0_policy[:, 1, 1, point].T[~np.isnan(m0_policy[:, 1, 1, point])], policy[point][1].T[1])


In [ ]:
# OLD ARRAY VERSION
# Initialize containers

# Container for endogenous gridpoints of (beginning-of-period) assets
# and corresponding consumption
policy = np.full((ngridm + 1, 2, 2, Tbar), np.nan)

# Value functions
value = np.full((ngridm + 1, 2, 2, Tbar), np.nan)
# Handling of last period and first elements
# policy
policy[1:, 0, 0, Tbar-1] = savingsgrid
policy[1:, 0, 1, Tbar-1] = savingsgrid
policy[1:, 1, :, Tbar-1] = policy[1:, 0, :, Tbar-1]
policy[0, :, :, :] = 0.00
# value
value[2:, 0, :, Tbar-1] = util(policy[2:, 0, :, Tbar-1], 0, theta, duw)
value[2:, 1, :, Tbar-1] = util(policy[2:, 0, :, Tbar-1], 1, theta, duw)
value[0:2, :, :, Tbar -1] = 0.00
value[0, 0, :, :] = 0.00